In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install langchain
!pip install pypdf
!pip install unstructured
!pip install sentence_transformers
!pip install pinecone-client
!pip install huggingface_hub
!pip install -U flask-cors
!ngrok config add-authtoken 2VCFw0HKXnTygcSKqVlrNfmy7MA_522AWhDWGypkY4bXeQccM
!pip install pyngrok
!pip install -q langchain transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.4/277.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.1/275.1 kB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 50.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.5 MB/s eta 0:00:00
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993224 sha256=12e6438423e2b46828c37d033b76709a797d62e5476a557b6346102f25c0b1ef
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a

In [ ]:
import firebase_admin
from firebase_admin import storage,initialize_app,credentials
from langchain.document_loaders import PyPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer
from langchain.chains.question_answering import load_qa_chain
import pinecone
import os
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from langchain.chains import LLMChain,SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap
import sys
#import firebase

In [ ]:
cred_object = cred = credentials.Certificate({
  """
  Firebase cred object

  """
})
databaseURL = "firebase database url"
initialize_app(cred_object, {
	'storageBucket':databaseURL
	})

In [ ]:
datadir = '/content/Test'
source_blob_name = "FILENAME"

In [ ]:
file_storage_uri = "https://storage.googleapis.com/page-talk.appspot.com/e3d0273a-3074-4384-be01-d2912363e0c9_Session-Hijacking.pdf?GoogleAccessId=firebase-adminsdk-xfipa%40page-talk.iam.gserviceaccount.com&Expires=1703186237&Signature=iBn5ifmF131B4%2FPQyWPX7a1P%2FIcaV1%2FOQ9mIAMffHgCDfBjRJfMlT6gKG%2BkWo6AdnhdbJ9aAwWqAWGFuClcvhSHGdMPg8u88wmTCAG0eK%2Bja2nA%2FOA%2FFodPYVZIZXIFudYAkzSjTf6OUgRLvmOPG8l08CQKR%2FMbKZeWXoALMquGMBUVqVvJqEG%2BeCiGXClNRumAgcIED0QgPfhaDC5uwPL5KxcWkACow%2BzgML0ZTTP9zn%2FOmuyTThZo01Dz6nqlhMunrwTGfdG8Q0Q52ALh5BZAM%2FVbMa%2F70aZbe4R%2BnzieYpO6AQeIgBBTjztCb5OTalh41ln7VcLd30UJdXnUMgQ%3D%3D"
file_path = file_storage_uri.split("/", 3)[-1].split("/", 1)[1].split("?")[0]
bucket_name = file_storage_uri.split("/", 3)[-1].split("/", 1)[0]
#bucket_name, file_path = file_storage_uri.split("/", 3)[-1].split("/", 1)
print(bucket_name,file_path)

page-talk.appspot.com e3d0273a-3074-4384-be01-d2912363e0c9_Session-Hijacking.pdf


In [ ]:
#initialize_app(cred_object,{"storageBucket":"https://console.firebase.google.com/project/page-talk/storage/page-talk.appspot.com/files"})
bucket = storage.bucket()
blob = bucket.blob(file_path)
local_fileName = "/content/test4.pdf"
blob.download_to_filename(local_fileName)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
# loader = PyPDFLoader("/content/test4.pdf")
# data = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap =0)
# docs = text_splitter.split_documents(data)

# embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')

# pinecone.init(
#     api_key = 'eb730e9d-c9de-4a50-945f-10622330f980',
#     environment= 'gcp-starter'
# )
# index_name = 'idp'

# docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name = index_name)



In [ ]:
pinecone.init(
            api_key = 'PINECONE API KEY',
            environment= 'gcp-starter'
        )

In [ ]:
model = AutoModelForCausalLM.from_pretrained("NousResearch/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             load_in_4bit=True,
                                             bnb_4bit_quant_type="nf4",
                                             bnb_4bit_compute_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [ ]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})
from langchain.chains.question_answering import  load_qa_chain

#prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = load_qa_chain(llm, chain_type = "stuff")

In [ ]:
!pip install flask-ngrok


In [ ]:
from flask import Flask,jsonify, request
from flask_ngrok import run_with_ngrok
from flask_cors import CORS
import logging
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)
@app.route('/post_data', methods=['POST'])
def post_data():
    if request.method == 'POST':
        data = request.json
        print("Received data:", data)
        print(data.get("exists"))
        index_name = 'idp'
        querry = data.get("querry")
        print(querry)
        if data.get("exists") == "False":
            print("Embeddings does not exist")
            file_storage_uri = data.get("content")
            print(file_storage_uri)
            file_path = file_storage_uri.split("/", 3)[-1].split("/", 1)[1].split("?")[0]
            bucket_name = file_storage_uri.split("/", 3)[-1].split("/", 1)[0]
            bucket = storage.bucket()
            blob = bucket.blob(file_path)
            local_fileName = "/content/test4.pdf"
            blob.download_to_filename(local_fileName)
            loader = PyPDFLoader("/content/test4.pdf")
            data = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500,chunk_overlap =0)
            docs = text_splitter.split_documents(data)
            docsearch = Pinecone.from_texts([t.page_content for t in docs], embeddings, index_name = index_name)
        else:
            docsearch = Pinecone.from_existing_index(index_name,embeddings)

    docs = docsearch.similarity_search(querry)

    response = llm_chain.run(input_documents = docs, question = querry)



    return jsonify({"response":response})





if __name__ == '__main__':
    app.run()

DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://66e1-34-124-136-29.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
DEBUG:flask_cors.extension:CORS have been already evaluated, skipping
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 04:57:54] "GET / HTTP/1.1" 404 -


...

DEBUG:flask_cors.extension:Request to '/post_data' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 05:03:57] "POST /post_data HTTP/1.1" 200 -
DEBUG:flask_cors.extension:Request to '/post_data' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': Tru

Received data: {'exists': 'True', 'content': 'https://storage.googleapis.com/page-talk.appspot.com/e3d0273a-3074-4384-be01-d2912363e0c9_Session-Hijacking.pdf?GoogleAccessId=firebase-adminsdk-xfipa%40page-talk.iam.gserviceaccount.com&Expires=1703186237&Signature=iBn5ifmF131B4%2FPQyWPX7a1P%2FIcaV1%2FOQ9mIAMffHgCDfBjRJfMlT6gKG%2BkWo6AdnhdbJ9aAwWqAWGFuClcvhSHGdMPg8u88wmTCAG0eK%2Bja2nA%2FOA%2FFodPYVZIZXIFudYAkzSjTf6OUgRLvmOPG8l08CQKR%2FMbKZeWXoALMquGMBUVqVvJqEG%2BeCiGXClNRumAgcIED0QgPfhaDC5uwPL5KxcWkACow%2BzgML0ZTTP9zn%2FOmuyTThZo01Dz6nqlhMunrwTGfdG8Q0Q52ALh5BZAM%2FVbMa%2F70aZbe4R%2BnzieYpO6AQeIgBBTjztCb5OTalh41ln7VcLd30UJdXnUMgQ%3D%3D', 'querry': 'Give me a summary'}
True
Give me a summary


DEBUG:flask_cors.extension:Request to '/post_data' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 05:07:57] "POST /post_data HTTP/1.1" 200 -
DEBUG:flask_cors.extension:Request to '/post_data' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': Tru

Received data: {'exists': 'True', 'content': 'https://storage.googleapis.com/page-talk.appspot.com/e3d0273a-3074-4384-be01-d2912363e0c9_Session-Hijacking.pdf?GoogleAccessId=firebase-adminsdk-xfipa%40page-talk.iam.gserviceaccount.com&Expires=1703186237&Signature=iBn5ifmF131B4%2FPQyWPX7a1P%2FIcaV1%2FOQ9mIAMffHgCDfBjRJfMlT6gKG%2BkWo6AdnhdbJ9aAwWqAWGFuClcvhSHGdMPg8u88wmTCAG0eK%2Bja2nA%2FOA%2FFodPYVZIZXIFudYAkzSjTf6OUgRLvmOPG8l08CQKR%2FMbKZeWXoALMquGMBUVqVvJqEG%2BeCiGXClNRumAgcIED0QgPfhaDC5uwPL5KxcWkACow%2BzgML0ZTTP9zn%2FOmuyTThZo01Dz6nqlhMunrwTGfdG8Q0Q52ALh5BZAM%2FVbMa%2F70aZbe4R%2BnzieYpO6AQeIgBBTjztCb5OTalh41ln7VcLd30UJdXnUMgQ%3D%3D', 'querry': 'Talk in detail about threat identification'}
True
Talk in detail about threat identification


DEBUG:flask_cors.extension:Request to '/post_data' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', '*')])
INFO:werkzeug:127.0.0.1 - - [30/Oct/2023 05:29:24] "POST /post_data HTTP/1.1" 200 -
